In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [0]:
root_loc = '/Volumes/workspace/default/tos_data'

schema = StructType([
    StructField("Date", StringType(), False),
    StructField("Action",StringType(), True),
    StructField("Symbol", StringType(), True),
    StructField("Description", StringType(), True),
    StructField("Quantity", StringType(), True),
    StructField("Price", StringType(), True),
    StructField("Fees & Comm", StringType(), True),
    StructField("Amount", StringType(), True)
    ])

In [0]:
df = (
  spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("rescuedDataColumn", "_rescued_data")
    .option("header", "true")
    .option("sep", ",")
    .schema(schema)
    .load(root_loc)
)
df_clean = df.selectExpr(
    "`Date` as Transaction_Date",
    "`Action` as Action",
    "`Symbol` as Symbol",
    "`Description` as Description",
    "`Quantity` as Quantity",
    "`Price` as Price",
    "`Fees & Comm` as Fees_Comm",
    "`Amount` as Amount",
    '_rescued_data'
)

In [0]:
(
  df_clean.writeStream
    .option("checkpointLocation", "/Volumes/workspace/default/checkpoints/tos_data_checkpoint")
    .trigger(availableNow=True)
    .toTable("workspace.default.tos_raw_data")
)